<a href="https://colab.research.google.com/github/kbg8146/AI_based_smart_nametag/blob/survey_smartnametag/smartnametag_survey3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ 1. 설치 및 인증
!pip install --upgrade gspread gspread_dataframe oauth2client
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
from collections import defaultdict

# ✅ 2. Google Sheets 연결 및 데이터 불러오기
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1kkt336f1G-XqfDuwCUOnqpKlxTcnwLQy-XS4SQv6lM0/edit"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.sheet1
data = worksheet.get_all_values()
headers = data[0]
rows = data[1:]
df = pd.DataFrame(rows, columns=headers)

# ✅ 3. 추천 산업군/직무: 수강 과목 키워드가 2개 이상일 경우에만
course_recommend_map = {
    "임베디드 시스템": ["마이크로프로세서", "SoC설계", "네트워크기반제어", "무선공학설계"],
    "반도체 설계": ["SoC설계", "전자회로", "디지털회로설계"],
    "반도체 공정": ["집적회로", "집적회로공정", "전자소자"],
    "AI 개발": ["딥러닝", "인공지능"],
    "시스템 소프트웨어": ["운영체제"],
    "신호처리": ["신호및시스템", "통신이론"],
    "통신 장비": ["무선공학", "안테나"],
    "디스플레이": ["기초디스플레이공학"]
}

def recommend_by_count(course_str):
    recommendations = []
    for field, keywords in course_recommend_map.items():
        count = sum(1 for k in keywords if k in course_str)
        if count >= 2:
            recommendations.append(field)
    return ", ".join(recommendations) if recommendations else "해당 없음"

df["추천 산업군/직무"] = df["수강 한 전공 과목 (복수 응답 가능)"].apply(recommend_by_count)

# ✅ 4. 그룹 분류 조건
group_conditions = {
    "임베디드+면접스터디": ["임베디드", "면접스터디"],
    "임베디드+팀프로젝트": ["임베디드", "팀프로젝트"],
    "방산+네트워킹": ["방산", "네트워킹"],
    "반도체+흥미": ["반도체", "흥미"]
}

# ✅ 5. 그룹 분류 함수
def assign_groups_filtered(row):
    purposes = [p.strip() for p in row["참가 목적 (복수 응답 가능)"].split(",")]
    non_interest_purposes = [p for p in purposes if "흥미" not in p]
    if not non_interest_purposes:
        return "기타"
    combined = row["관심 분야 (복수 응답 가능)"] + "," + row["참가 목적 (복수 응답 가능)"]
    matched = [group for group, keys in group_conditions.items()
               if all(k in combined for k in keys)]
    return ", ".join(matched) if matched else "기타"

df["자동분류그룹"] = df.apply(assign_groups_filtered, axis=1)

# ✅ 6. 그룹당 최대 5명으로 조 편성
group_map = defaultdict(list)
for _, row in df.iterrows():
    for group in row["자동분류그룹"].split(", "):
        group_map[group].append(row["이름"])

group_chunked_data = []
for group_name, members in group_map.items():
    if group_name == "기타":
        continue
    for i in range(0, len(members), 5):
        chunk = members[i:i+5]
        numbered_group = f"{group_name} {i//5 + 1}조" if len(members) > 5 else group_name
        for m in chunk:
            others = [o for o in chunk if o != m]
            group_chunked_data.append([
                m,
                numbered_group,
                ", ".join(others)
            ])

# ✅ 7. 결과 DataFrame 구성 및 저장
result_df = pd.DataFrame(group_chunked_data, columns=["참가자", "소속 그룹", "같은 그룹 참가자"])
result_df["추천 산업군/직무"] = result_df["참가자"].map(
    df.set_index("이름")["추천 산업군/직무"].to_dict()
)

# ✅ 8. Google Sheets에 Sheet2로 저장
try:
    result_ws = sh.worksheet("Sheet2")
except:
    result_ws = sh.add_worksheet(title="Sheet2", rows="100", cols="20")

set_with_dataframe(result_ws, result_df)
